In [ ]:
!pip install selenium
# !apt-get update 
# !apt install chromium-chromedriver
!pip install scrapy

     |████████████████████████████████| 958 kB 5.1 MB/s 
     |████████████████████████████████| 138 kB 28.1 MB/s 
     |████████████████████████████████| 356 kB 60.8 MB/s 
     |████████████████████████████████| 3.6 MB 52.8 MB/s 
     |████████████████████████████████| 55 kB 3.4 MB/s 
     |████████████████████████████████| 58 kB 5.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.8 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 254 kB 4.9 MB/s 
     |████████████████████████████████| 65 kB 3.5 MB/s 
 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import requests
import scrapy
from scrapy import Selector

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from time import sleep
from datetime import datetime
import pandas as pd
import numpy as np
import requests
import openpyxl
from bs4 import BeautifulSoup
import pandas as pd
import urllib.request
import re
import math 

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
SAVING_PATH = "/content/drive/MyDrive/Colab Notebooks/Hackathon/"

In [ ]:
def get_airlines_url():
  html = requests.get('https://www.seatguru.com/browseairlines/browseairlines.php').content
  sel_SG = Selector(text = html)
  name_list  = sel_SG.css('div.browseAirlines li > a::text').extract()
  url_list_SG = sel_SG.css('div.browseAirlines li > a::attr(href)').extract()
  return name_list,url_list_SG

In [ ]:
def get_link_df(sg_links_df, col_list):
  for i in range(len(name_list)):
    geturl = SG_URL + url_list_SG[i]
    html = requests.get(geturl).content
    sel = Selector(text = html)
    planes = sel.css('div.aircraft_seats > a::text').extract()
    planes_link = sel.css('div.aircraft_seats > a::attr(href)').extract()
    sg_links_df = pd.concat([sg_links_df, pd.DataFrame({'Airlines': [name_list[i]]*len(planes), 
                                                                  'main_link':[url_list_SG[i]]*len(planes), 
                                                                  'Plane_type':planes,
                                                                  'pt_link':planes_link},
                                                                  columns = col_list)],ignore_index = True)
  return sg_links_df


In [ ]:
def get_final_df(sg_links_df,sg_df):
    print(sg_links_df)
    for index,row in sg_links_df.iterrows():
      reviews_url = SG_URL + row['pt_link']
      print(index,row['pt_link'])
      html = requests.get(reviews_url).content
      sel = Selector(text = html)
      seat_date = sel.css('div.submitted > span:nth-of-type(3)::text').extract()
      comment = sel.css('div.comment::text').extract()
      if not (len(seat_date)==0 and len(comment)==0):
        num_rows = max(len(seat_date)==0,len(comment)==0)
        sg_df = pd.concat([sg_df, pd.DataFrame({'Airlines': row['Airlines'] , 
                                                          'main_link': row['main_link'] , 
                                                          'Plane_type': row['Plane_type'],
                                                          'pt_link': row['pt_link'] ,
                                                          'date_seat': seat_date,
                                                          'comment':comment},
                                                          columns = col_list2)],ignore_index = True)
    #print(type(sg_df_final))
    return sg_df

In [ ]:
SG_URL = 'https://www.seatguru.com'
col_list = ['Airlines','main_link','Plane_type','pt_link']
sg_links_df = pd.DataFrame(columns = col_list)
name_list, url_list_SG = get_airlines_url()
#print(name_list)
#print(url_list_SG)
sg_links_df =  get_link_df(sg_links_df,col_list)
#print(type(sg_links_df))
col_list2 = ['Airlines','main_link','Plane_type','pt_link','date_seat','comment']
sg_df = pd.DataFrame(columns = col_list2)
sg_df_final = get_final_df(sg_links_df,sg_df)
#sg_df_final.to_csv(OUTPUT_FILE)

             Airlines  ...                                            pt_link
0     Aegean Airlines  ...  /airlines/Aegean_Airlines/Aegean_Airlines_Airb...
1     Aegean Airlines  ...  /airlines/Aegean_Airlines/Aegean_Airlines_Airb...
2     Aegean Airlines  ...  /airlines/Aegean_Airlines/Aegean_Airlines_Airb...
3     Aegean Airlines  ...  /airlines/Aegean_Airlines/Aegean_Airlines_ATR_...
4     Aegean Airlines  ...  /airlines/Aegean_Airlines/Aegean_Airlines_Bomb...
...               ...  ...                                                ...
1273  Xiamen Airlines  ...  /airlines/Xiamen_Airlines/Xiamen_Airlines_Boei...
1274  Xiamen Airlines  ...  /airlines/Xiamen_Airlines/Xiamen_Airlines_Boei...
1275  Xiamen Airlines  ...  /airlines/Xiamen_Airlines/Xiamen_Airlines_Boei...
1276  Xiamen Airlines  ...  /airlines/Xiamen_Airlines/Xiamen_Airlines_Boei...
1277  Xiamen Airlines  ...  /airlines/Xiamen_Airlines/Xiamen_Airlines_Boei...

[1278 rows x 4 columns]
0 /airlines/Aegean_Airlines/Aegean_Airl

In [ ]:
print(sg_df_final)

              Airlines  ...                                            comment
0      Aegean Airlines  ...  \n                                            ...
1      Aegean Airlines  ...  \n                                            ...
2      Aegean Airlines  ...  \n                                            ...
3      Aegean Airlines  ...  \n                                            ...
4      Aegean Airlines  ...  \n                                            ...
...                ...  ...                                                ...
17335          Wizzair  ...  \n                                            ...
17336          Wizzair  ...  \n                                            ...
17337  Xiamen Airlines  ...  \n                                            ...
17338  Xiamen Airlines  ...  \n                                            ...
17339  Xiamen Airlines  ...  \n                                            ...

[17340 rows x 6 columns]


In [ ]:
sg_df_final.to_csv(SAVING_PATH + "sg.csv")